In [90]:
pip install pandas 


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\fares mohamed\appdata\local\programs\python\python312\lib\site-packages (2.1.4)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

# Load the data

In [93]:
train_file_path = "D:\\Pattern Recognition\\adult.data"
test_file_path = "D:\\Pattern Recognition\\adult.test"
# Define column names
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", 
           "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", 
           "hours-per-week", "native-country", "income"]

train_data = pd.read_csv(train_file_path, header=None, names=columns, na_values=" ?")
test_data = pd.read_csv(test_file_path, header=1, names=columns, na_values=" ?") # Skip row 1 

print(train_data.head())
print("\n")
print("Shape of train_data before dropping:", train_data.shape)
print("......................................................................................")
print(test_data.head())
print("\n")
print("Shape of test_data before dropping:", test_data.shape)


   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States   <=50

# Drop rows with missing values

In [94]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

# Output shape after dropping
print("Shape of train_data after dropping:", train_data.dropna().shape)
print("Shape of test_data after dropping:", test_data.dropna().shape)



Shape of train_data after dropping: (30162, 15)
Shape of test_data after dropping: (15059, 15)


# One-hot encode categorical variables

In [95]:

# Convert income column to binary
train_data["income"] = train_data["income"].apply(lambda x: 1 if x == ' >50K' else 0)
test_data["income"] = test_data["income"].apply(lambda x: 1 if x == ' >50K.' else 0)

# One-hot encode 
combined_data = pd.concat([train_data, test_data])
combined_data = pd.get_dummies(combined_data, columns=["workclass", "education", "marital-status", 
                                                       "occupation", "relationship", "race", "sex", "native-country"])

# Split the combined data back into training and testing sets
X_train = combined_data[:len(train_data)].drop("income", axis=1)
y_train = combined_data[:len(train_data)]["income"]
X_test = combined_data[len(train_data):].drop("income", axis=1)
y_test = combined_data[len(train_data):]["income"]

# Display the shapes of training and testing data
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (30162, 104)
Shape of y_train: (30162,)
Shape of X_test: (15059, 104)
Shape of y_test: (15059,)


# Compute Sensitivity and Specificity

In [96]:
# Train Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Predict income level for testing data
y_pred = nb_classifier.predict(X_test)

# Compute Sensitivity and Specificity
conf_matrix = confusion_matrix(y_test, y_pred)

# Check if the confusion matrix has multiple rows (indicating predictions for both classes)
if conf_matrix.shape[0] < 2:
    print("Error: Confusion matrix has only one row, indicating predictions for only one class.")
else:
    # Extract values from confusion matrix
    TP = conf_matrix[1, 1]
    FP = conf_matrix[0, 1]
    TN = conf_matrix[0, 0]
    FN = conf_matrix[1, 0]
 
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)

    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

Sensitivity: 0.3062162162162162
Specificity: 0.9458579100272911


# Compute the posterior probability of making over 50K a year

In [97]:
# Predict probabilities for testing data
posterior_probs = nb_classifier.predict_proba(X_test)

# Extract the probability of the positive class (making over 50K a year)
positive_class_probs = posterior_probs[:, 1]

# Print the posterior probabilities
print("Posterior Probabilities of making over 50K a year:")
print(positive_class_probs)

Posterior Probabilities of making over 50K a year:
[0.01595412 0.00665536 1.         ... 0.02297512 0.99999491 0.02961601]
